In [3]:
%load_ext autoreload
%autoreload 2

from utils import (
    print_rich,
    create_transaction,
    get_all_inventory,
    get_stock_level,
    get_supplier_delivery_date,
    get_cash_balance,
    generate_financial_report,
    search_quote_history,
)

import json, re
import pandas as pd
from datetime import datetime
from typing import Dict,List

In [4]:
from sqlalchemy import inspect
from sqlalchemy import create_engine, Engine
from helpers import db_engine, state

inspector = inspect(db_engine)
print(inspector.get_table_names())


['inventory', 'quote_requests', 'quotes', 'transactions']


In [14]:
for table in inspector.get_table_names():
    print(table)
    df = pd.read_sql(f"SELECT * FROM {table}", db_engine)  #LIMIT 10
    display(df)

inventory


,item_name,category,unit_price,current_stock,min_stock_level
0,Paper plates,product,0.10,748,144
1,100 lb cover stock,specialty,0.50,636,85
2,Glossy paper,paper,0.20,587,147
3,Rolls of banner paper (36-inch width),large_format,2.50,546,65
4,Photo paper,paper,0.25,423,58
5,Cardstock,paper,0.15,595,148
6,Colored paper,paper,0.10,788,143
7,80 lb text paper,specialty,0.40,249,91
8,Large poster paper (24x36 inches),large_format,1.00,699,89
9,Table covers,product,1.50,736,77


quote_requests


,mood,job,need_size,event,response,id
0,stressed,event manager,large,meeting,I would like to request a large order of high-...,1
1,pissed off,school board resouorce manager,large,meeting,I need to order 10 reams of standard copy pape...,2
2,miserable,event manager,small,festival,I would like to place an order for 500 sheets ...,3
3,pissed off,restaurant manager,small,concert,I need to order 1000 sheets of high-quality A4...,4
4,pissed off,school board resouorce manager,large,assembly,"I need to order 500 sheets of A4 paper, 300 sh...",5
...,...,...,...,...,...,...
95,stressed,business owner,medium,gathering,I would like to place a medium order for the f...,96
96,sad,law firm receptionist,medium,celebration,I would like to place a medium order for the f...,97
97,stressed,school principal,medium,party,I would like to place a medium order for the f...,98
98,pissed off,hotel manager,medium,parade,I would like to place a medium order for the f...,99


quotes


,request_id,total_amount,quote_explanation,order_date,job_type,order_size,event_type
0,1,96,Thank you for your large order! We have calcul...,2025-01-01T00:00:00,event manager,large,meeting
1,2,60,For your order of 10 reams of standard copy pa...,2025-01-01T00:00:00,school board resouorce manager,large,meeting
2,3,780,Thank you for your order! For the 500 sheets o...,2025-01-01T00:00:00,event manager,small,festival
3,4,120,Thank you for your order! For 1000 sheets of h...,2025-01-01T00:00:00,restaurant manager,small,concert
4,5,122,Thank you for your order! For the upcoming ass...,2025-01-01T00:00:00,school board resouorce manager,large,assembly
...,...,...,...,...,...,...,...
95,96,80,Thank you for your order! For your upcoming ga...,2025-01-01T00:00:00,business owner,medium,gathering
96,97,300,Thank you for your order! For the supplies req...,2025-01-01T00:00:00,law firm receptionist,medium,celebration
97,98,126,Thank you for your order for the upcoming part...,2025-01-01T00:00:00,school principal,medium,party
98,99,189,Thank you for your order! I have calculated th...,2025-01-01T00:00:00,hotel manager,medium,parade


transactions


,id,item_name,transaction_type,units,price,transaction_date
0,0.0,None,sales,NaN,50000.0,2025-01-01T00:00:00
1,1.0,Paper plates,stock_orders,748.0,74.8,2025-01-01T00:00:00
2,2.0,100 lb cover stock,stock_orders,636.0,318.0,2025-01-01T00:00:00
3,3.0,Glossy paper,stock_orders,587.0,117.4,2025-01-01T00:00:00
4,4.0,Rolls of banner paper (36-inch width),stock_orders,546.0,1365.0,2025-01-01T00:00:00
...,...,...,...,...,...,...
85,85.0,100 lb cover stock,sales,5000.0,2500.0,2025-05-15
86,86.0,Invitation cards,stock_orders,9623.0,4811.5,2025-04-17
87,87.0,Invitation cards,sales,10000.0,5000.0,2025-05-15
88,88.0,Large poster paper (24x36 inches),stock_orders,2400.0,2400.0,2025-04-17


In [4]:
help(get_all_inventory)

Help on SimpleTool in module smolagents.tools object:

class SimpleTool(Tool)
 |  Method resolution order:
 |      SimpleTool
 |      Tool
 |      BaseTool
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  forward = get_all_inventory(self, as_of_date: str, db_engine: sqlalchemy.engine.base.Engine = Engine(sqlite:///beaver.db)) -> Dict[str, int]
 |      Retrieve a snapshot of available inventory as of a specific date.
 |      
 |      This function calculates the net quantity of each item by summing 
 |      all stock orders and subtracting all sales up to and including the given date.
 |      
 |      Only items with positive stock are included in the result.
 |      
 |      Args:
 |          as_of_date (str): ISO-formatted date string (YYYY-MM-DD) representing the inventory cutoff.
 |          db_engine (Engine): sqlite

In [15]:
get_all_inventory(as_of_date="2025-04-30")

{'100 lb cover stock': 5085.0,
 '80 lb text paper': 91.0,
 'A4 paper': 135.0,
 'Banner paper': 793.0,
 'Butcher paper': 365.0,
 'Cardstock': 148.0,
 'Crepe paper': 104.0,
 'Glossy paper': 147.0,
 'Invitation cards': 10149.0,
 'Kraft paper': 443.0,
 'Large poster paper (24x36 inches)': 2089.0,
 'Paper plates': 748.0,
 'Patterned paper': 548.0,
 'Photo paper': 423.0,
 'Presentation folders': 389.0,
 'Rolls of banner paper (36-inch width)': 546.0,
 'Table covers': 736.0}

In [6]:
help(get_stock_level)

Help on SimpleTool in module smolagents.tools object:

class SimpleTool(Tool)
 |  Method resolution order:
 |      SimpleTool
 |      Tool
 |      BaseTool
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  forward = get_stock_level(self, item_name: str, as_of_date: Union[str, datetime.datetime], db_engine: sqlalchemy.engine.base.Engine = Engine(sqlite:///beaver.db)) -> pandas.core.frame.DataFrame
 |      Retrieve the stock level of a specific item as of a given date.
 |      
 |      This function calculates the net stock by summing all 'stock_orders' and 
 |      subtracting all 'sales' transactions for the specified item up to the given date.
 |      
 |      Args:
 |          item_name (str): The name of the item to look up.
 |          as_of_date (str or datetime): The cutoff date (inclusive) for calculating stock.
 |

In [16]:
get_stock_level(item_name="Paper plates", as_of_date="2025-04-30")

,item_name,current_stock
0,Paper plates,748.0


In [22]:
print(json.dumps(generate_financial_report(as_of_date="2025-01-02"), indent=4))

{
    "as_of_date": "2025-01-02",
    "cash_balance": 45059.7,
    "inventory_value": 4940.299999999999,
    "total_assets": 50000.0,
    "inventory_summary": [
        {
            "item_name": "Paper plates",
            "stock": 748.0,
            "unit_price": 0.1,
            "value": 74.8
        },
        {
            "item_name": "100 lb cover stock",
            "stock": 636.0,
            "unit_price": 0.5,
            "value": 318.0
        },
        {
            "item_name": "Glossy paper",
            "stock": 587.0,
            "unit_price": 0.2,
            "value": 117.4
        },
        {
            "item_name": "Rolls of banner paper (36-inch width)",
            "stock": 546.0,
            "unit_price": 2.5,
            "value": 1365.0
        },
        {
            "item_name": "Photo paper",
            "stock": 423.0,
            "unit_price": 0.25,
            "value": 105.75
        },
        {
            "item_name": "Cardstock",
            "stock"

In [30]:
get_cash_balance(as_of_date="2025-01-02")

45059.7

In [24]:
report = generate_financial_report("2025-01-02")
print(report)

{'as_of_date': '2025-01-02', 'cash_balance': 45059.7, 'inventory_value': np.float64(4940.299999999999), 'total_assets': np.float64(50000.0), 'inventory_summary': [{'item_name': 'Paper plates', 'stock': np.float64(748.0), 'unit_price': 0.1, 'value': np.float64(74.8)}, {'item_name': '100 lb cover stock', 'stock': np.float64(636.0), 'unit_price': 0.5, 'value': np.float64(318.0)}, {'item_name': 'Glossy paper', 'stock': np.float64(587.0), 'unit_price': 0.2, 'value': np.float64(117.4)}, {'item_name': 'Rolls of banner paper (36-inch width)', 'stock': np.float64(546.0), 'unit_price': 2.5, 'value': np.float64(1365.0)}, {'item_name': 'Photo paper', 'stock': np.float64(423.0), 'unit_price': 0.25, 'value': np.float64(105.75)}, {'item_name': 'Cardstock', 'stock': np.float64(595.0), 'unit_price': 0.15, 'value': np.float64(89.25)}, {'item_name': 'Colored paper', 'stock': np.float64(788.0), 'unit_price': 0.1, 'value': np.float64(78.80000000000001)}, {'item_name': '80 lb text paper', 'stock': np.float6

In [26]:
report.keys()

dict_keys(['as_of_date', 'cash_balance', 'inventory_value', 'total_assets', 'inventory_summary', 'top_selling_products'])

In [27]:
report['inventory_summary']

[{'item_name': 'Paper plates',
  'stock': np.float64(748.0),
  'unit_price': 0.1,
  'value': np.float64(74.8)},
 {'item_name': '100 lb cover stock',
  'stock': np.float64(636.0),
  'unit_price': 0.5,
  'value': np.float64(318.0)},
 {'item_name': 'Glossy paper',
  'stock': np.float64(587.0),
  'unit_price': 0.2,
  'value': np.float64(117.4)},
 {'item_name': 'Rolls of banner paper (36-inch width)',
  'stock': np.float64(546.0),
  'unit_price': 2.5,
  'value': np.float64(1365.0)},
 {'item_name': 'Photo paper',
  'stock': np.float64(423.0),
  'unit_price': 0.25,
  'value': np.float64(105.75)},
 {'item_name': 'Cardstock',
  'stock': np.float64(595.0),
  'unit_price': 0.15,
  'value': np.float64(89.25)},
 {'item_name': 'Colored paper',
  'stock': np.float64(788.0),
  'unit_price': 0.1,
  'value': np.float64(78.80000000000001)},
 {'item_name': '80 lb text paper',
  'stock': np.float64(249.0),
  'unit_price': 0.4,
  'value': np.float64(99.60000000000001)},
 {'item_name': 'Large poster paper (2

In [15]:
help(search_quote_history)

Help on function search_quote_history in module utils:

search_quote_history(search_terms: List[str], limit: int = 5, db_engine=Engine(sqlite:///beaver.db)) -> List[Dict]
    Retrieve a list of historical quotes that match any of the provided search terms.
    
    The function searches both the original customer request (from `quote_requests`) and
    the explanation for the quote (from `quotes`) for each keyword. Results are sorted by
    most recent order date and limited by the `limit` parameter.
    
    Args:
        search_terms (List[str]): List of terms to match against customer requests and explanations.
        limit (int, optional): Maximum number of quote records to return. Default is 5.
    
    Returns:
        List[Dict]: A list of matching quotes, each represented as a dictionary with fields:
            - original_request
            - total_amount
            - quote_explanation
            - job_type
            - order_size
            - event_type
            - or

In [29]:
search_quote_history(search_terms=['meeting','large','glossy paper'])

[]

In [32]:
get_supplier_delivery_date(input_date_str="2025-01-02", quantity=500)

FUNC (get_supplier_delivery_date): Calculating for qty 500 from date string '2025-01-02'


'2025-01-06'

In [33]:
help(create_transaction)

Help on function create_transaction in module utils:

create_transaction(item_name: str, transaction_type: str, quantity: int, price: float, date: Union[str, datetime.datetime], db_engine: Engine(sqlite:///beaver.db)) -> int
    This function records a transaction of type 'stock_orders' or 'sales' with a specified
    item name, quantity, total price, and transaction date into the 'transactions' table of the database.
    
    Args:
        item_name (str): The name of the item involved in the transaction.
        transaction_type (str): Either 'stock_orders' or 'sales'.
        quantity (int): Number of units involved in the transaction.
        price (float): Total price of the transaction.
        date (str or datetime): Date of the transaction in ISO 8601 format.
    
    Returns:
        int: The ID of the newly inserted transaction.
    
    Raises:
        ValueError: If `transaction_type` is not 'stock_orders' or 'sales'.
        Exception: For other database or execution error

In [43]:
create_transaction(
    item_name="Paper plates", 
    date="2025-01-03",
    quantity=100,
    transaction_type='stock_orders',
    price=100*10
)

20

In [45]:
for table in inspector.get_table_names():
    print(table)
    df = pd.read_sql(f"SELECT * FROM {table}", db_engine)
    display(df)

inventory


,item_name,category,unit_price,current_stock,min_stock_level
0,Paper plates,product,0.10,748,144
1,100 lb cover stock,specialty,0.50,636,85
2,Glossy paper,paper,0.20,587,147
3,Rolls of banner paper (36-inch width),large_format,2.50,546,65
4,Photo paper,paper,0.25,423,58
5,Cardstock,paper,0.15,595,148
6,Colored paper,paper,0.10,788,143
7,80 lb text paper,specialty,0.40,249,91
8,Large poster paper (24x36 inches),large_format,1.00,699,89
9,Table covers,product,1.50,736,77


quote_requests


,mood,job,need_size,event,response,id
0,stressed,event manager,large,meeting,I would like to request a large order of high-...,1
1,pissed off,school board resouorce manager,large,meeting,I need to order 10 reams of standard copy pape...,2
2,miserable,event manager,small,festival,I would like to place an order for 500 sheets ...,3
3,pissed off,restaurant manager,small,concert,I need to order 1000 sheets of high-quality A4...,4
4,pissed off,school board resouorce manager,large,assembly,"I need to order 500 sheets of A4 paper, 300 sh...",5
...,...,...,...,...,...,...
95,stressed,business owner,medium,gathering,I would like to place a medium order for the f...,96
96,sad,law firm receptionist,medium,celebration,I would like to place a medium order for the f...,97
97,stressed,school principal,medium,party,I would like to place a medium order for the f...,98
98,pissed off,hotel manager,medium,parade,I would like to place a medium order for the f...,99


quotes


,request_id,total_amount,quote_explanation,order_date,job_type,order_size,event_type
0,1,96,Thank you for your large order! We have calcul...,2025-01-01T00:00:00,event manager,large,meeting
1,2,60,For your order of 10 reams of standard copy pa...,2025-01-01T00:00:00,school board resouorce manager,large,meeting
2,3,780,Thank you for your order! For the 500 sheets o...,2025-01-01T00:00:00,event manager,small,festival
3,4,120,Thank you for your order! For 1000 sheets of h...,2025-01-01T00:00:00,restaurant manager,small,concert
4,5,122,Thank you for your order! For the upcoming ass...,2025-01-01T00:00:00,school board resouorce manager,large,assembly
...,...,...,...,...,...,...,...
95,96,80,Thank you for your order! For your upcoming ga...,2025-01-01T00:00:00,business owner,medium,gathering
96,97,300,Thank you for your order! For the supplies req...,2025-01-01T00:00:00,law firm receptionist,medium,celebration
97,98,126,Thank you for your order for the upcoming part...,2025-01-01T00:00:00,school principal,medium,party
98,99,189,Thank you for your order! I have calculated th...,2025-01-01T00:00:00,hotel manager,medium,parade


transactions


,id,item_name,transaction_type,units,price,transaction_date
0,None,None,sales,NaN,50000.00,2025-01-01T00:00:00
1,None,Paper plates,stock_orders,748.0,74.80,2025-01-01T00:00:00
2,None,100 lb cover stock,stock_orders,636.0,318.00,2025-01-01T00:00:00
3,None,Glossy paper,stock_orders,587.0,117.40,2025-01-01T00:00:00
4,None,Rolls of banner paper (36-inch width),stock_orders,546.0,1365.00,2025-01-01T00:00:00
5,None,Photo paper,stock_orders,423.0,105.75,2025-01-01T00:00:00
6,None,Cardstock,stock_orders,595.0,89.25,2025-01-01T00:00:00
7,None,Colored paper,stock_orders,788.0,78.80,2025-01-01T00:00:00
8,None,80 lb text paper,stock_orders,249.0,99.60,2025-01-01T00:00:00
9,None,Large poster paper (24x36 inches),stock_orders,699.0,699.00,2025-01-01T00:00:00


In [21]:
get_stock_level(item_name="Cardstock", as_of_date="2025-01-02")

,item_name,current_stock
0,Cardstock,595.0


In [22]:
get_stock_level(item_name="Cardstock", as_of_date="2025-04-30")

,item_name,current_stock
0,Cardstock,148.0


In [6]:
def assess_stock_item(item_name:str, quantity:int, input_date_str:str, db_engine: Engine=db_engine)->Dict:
    """
    calculate if a stock_order is required given current stock level and customer ordered quantity
    
    Args:
        item_name (str): The name of the item to look up.
        quantity (int): The number of units in the order.
        input_date_str (str): Date of the look up in ISO format (YYYY-MM-DD).
        db_engine (Engine): sqlite database client (default = db_engine)
     
    Returns:
        Dict[Dict[str,str], str]: A dictionary providing required stock_order and resulting delivery date.
    """
    # check current inventory level
    current_stock_level = get_stock_level(item_name, input_date_str).at[0,'current_stock']
    # get min stock level requirements
    df = pd.read_sql(f"SELECT * FROM inventory", db_engine).set_index('item_name')
    min_stock_level = df.at[item_name, 'min_stock_level']
    
    if (current_stock_level - quantity) >= min_stock_level:
        return {
            'stock_order':{},
            'delivery_date': input_date_str
        }

    else:
        quantity_to_purchase = min_stock_level + quantity - current_stock_level
        return {
            'stock_order':{'item_name':item_name, 'quantity':quantity_to_purchase},
            'delivery_date': get_supplier_delivery_date(input_date_str=input_date_str, quantity=quantity_to_purchase)
        }

In [8]:
assess_stock_item('Paper plates', 100, '2025-01-03')

KeyError: 'paper plates'

In [10]:
import re
import json
from datetime import datetime

# predefined list of allowed items
ALLOWED_ITEMS = {"Glossy paper", "Cardstock", "Colored paper", "Plain paper"}

def parse_agent_response(response: str):
    result = {"bill of materials":[{"item": None, "quantity": None}], "delivery_date": None, "comment": None}

    # --- Extract delivery date ---
    date_match = re.search(r"delivery date:\s*([\d-]+)", response, re.IGNORECASE)
    if date_match:
        try:
            extracted_date = date_match.group(1)
            result["delivery_date"] = datetime.strptime(extracted_date, "%Y-%m-%d").date().isoformat()
        except ValueError:
            pass  # leave as None if invalid

    # --- Extract comment ---
    comment_match = re.search(r"comment:\s*(.*)", response, re.IGNORECASE)
    if comment_match:
        comment = comment_match.group(1).strip()
        result["comment"] = comment if comment else None

    # --- Extract items ---
    # Split by commas but ignore parts with 'delivery date' or 'comment'
    parts = [p.strip() for p in response.split(",") if not re.search(r"(delivery date|comment)", p, re.IGNORECASE)]
    
    bill_of_materials = []
    for part in parts:
        # match "Item:Quantity"
        match = re.match(r"(.+?):\s*(\d+)", part)
        if match:
            item, qty = match.groups()
            item = item.strip()
            if item in ALLOWED_ITEMS:
                bill_of_materials.append({"item":item, "quantity":int(qty)})
                #break  # only one allowed
    
    if bill_of_materials:
        result["bill of materials"] = bill_of_materials

    return result


# Example usage
response = "Glossy paper:200, Cardstock:100, Colored paper:100, delivery date: 2025-04-15, comment:"
parsed = parse_agent_response(response)

print(json.dumps(parsed, indent=2))


{
  "bill of materials": [
    {
      "item": "Glossy paper",
      "quantity": 200
    },
    {
      "item": "Cardstock",
      "quantity": 100
    },
    {
      "item": "Colored paper",
      "quantity": 100
    }
  ],
  "delivery_date": "2025-04-15",
  "comment": null
}


In [6]:
# Example usage
response = "Glossy paper:200, delivery date: 2025-04-15, comment: eheh"
parsed = parse_agent_response(response)
print(json.dumps(parsed, indent=2))

{
  "item": "Glossy paper",
  "quantity": 200,
  "delivery_date": "2025-04-15",
  "comment": "eheh"
}


In [37]:
pd.read_sql(f"SELECT COUNT(*) FROM transactions", db_engine)

,COUNT(*)
0,23


In [44]:
pd.read_sql(f"SELECT * FROM transactions", db_engine)

,id,item_name,transaction_type,units,price,transaction_date
0,0.0,None,sales,NaN,50000.00,2025-01-01T00:00:00
1,1.0,Paper plates,stock_orders,748.0,74.80,2025-01-01T00:00:00
2,2.0,100 lb cover stock,stock_orders,636.0,318.00,2025-01-01T00:00:00
3,3.0,Glossy paper,stock_orders,587.0,117.40,2025-01-01T00:00:00
4,4.0,Rolls of banner paper (36-inch width),stock_orders,546.0,1365.00,2025-01-01T00:00:00
5,5.0,Photo paper,stock_orders,423.0,105.75,2025-01-01T00:00:00
6,6.0,Cardstock,stock_orders,595.0,89.25,2025-01-01T00:00:00
7,7.0,Colored paper,stock_orders,788.0,78.80,2025-01-01T00:00:00
8,8.0,80 lb text paper,stock_orders,249.0,99.60,2025-01-01T00:00:00
9,9.0,Large poster paper (24x36 inches),stock_orders,699.0,699.00,2025-01-01T00:00:00


In [22]:
# Month names pattern for natural-language dates
MONTHS = r'(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|' \
         r'Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:t(?:ember)?)?|' \
         r'Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)'

def count_order_quantities(text: str):
    """
    Return (count, quantities_list) where quantities_list are integers that
    represent ordered item quantities. The function:
      - accepts numbers with thousands separators (e.g., 10,000)
      - ignores numbers inside dates (ISO-like and 'Month Day, Year' forms)
      - ignores digits attached to letters (e.g., A4, A3, X-12)
    """
    exclude_spans = []



    # --- Exclude dates (collect their spans) ---
    # ISO-like YYYY-MM-DD or YYYY/MM/DD
    for m in re.finditer(r'\b\d{4}[-/]\d{1,2}[-/]\d{1,2}\b', text):
        exclude_spans.append(m.span())

    # Year-Month (no day), e.g., 2025-04 or 2025/04
    for m in re.finditer(r'\b\d{4}[-/]\d{1,2}\b', text):
        exclude_spans.append(m.span())

    # Month Day, Year  (e.g., April 15, 2025)  and  'Apr 15 2025'
    for m in re.finditer(rf'\b{MONTHS}\s+\d{{1,2}}(?:st|nd|rd|th)?(?:,\s*|\s+)\d{{4}}\b',
                         text, flags=re.IGNORECASE):
        exclude_spans.append(m.span())

    # Day Month Year  (e.g., 15 April 2025)
    for m in re.finditer(rf'\b\d{{1,2}}(?:st|nd|rd|th)?\s+{MONTHS}\s+\d{{4}}\b',
                         text, flags=re.IGNORECASE):
        exclude_spans.append(m.span())

    # Month Year  (e.g., April 2025)
    for m in re.finditer(rf'\b{MONTHS}\s+\d{{4}}\b', text, flags=re.IGNORECASE):
        exclude_spans.append(m.span())

    # Numeric M/D/Y or D/M/Y (e.g., 04/15/2025, 15/04/2025)
    for m in re.finditer(r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b', text):
        exclude_spans.append(m.span())

    # --- Exclude digits that are part of codes like A4, A3, X-12, etc. ---
    for m in re.finditer(r'\b[A-Za-z]+-\d+\b|\b[A-Za-z]+\d+[A-Za-z]*\b', text):
        s, e = m.span()
        sub = text[s:e]
        for d in re.finditer(r'\d+', sub):  # add only the digits inside that token
            exclude_spans.append((s + d.start(), s + d.end()))

    # --- Exclude decimals ---
    for m in re.finditer(r'\b\d+\.\d+\b', text):  # e.g., 3.5
        exclude_spans.append(m.span())

    # --- Exclude inches (24", 30”, 40in, 24', 30 lb) ---
    for m in re.finditer(r'\b\d+(?:,\d{3})*(?=\s*(?:"|”|\'|in\b))', text, flags=re.IGNORECASE):
        exclude_spans.append(m.span())

    # --- Exclude percentages (50%) ---
    for m in re.finditer(r'\b\d+(?:,\d{3})*(?=%)', text):
        exclude_spans.append(m.span())

    # --- Exclude weight/length units (30 lb, 20 kg, 5 g, 10 oz, etc.) ---
    for m in re.finditer(r'\b\d+(?:,\d{3})*\s*(lb|lbs|kg|g|oz|mg|m|cm|mm)\b',
                         text, flags=re.IGNORECASE):
        exclude_spans.append(m.span())

    def is_in_excluded(span):
        a, b = span
        for x, y in exclude_spans:
            if a < y and b > x:  # overlaps any excluded region
                return True
        return False

    # --- Pick candidate numbers (not adjacent to letters), allow 1,234,567 ---
    quantities = []
    for m in re.finditer(r'(?<![A-Za-z])(\d{1,3}(?:,\d{3})+|\d+)(?![A-Za-z])', text):
        if not is_in_excluded(m.span()):
            quantities.append(int(m.group(1).replace(',', '')))

    return len(quantities), quantities


# --- Example usage ---
order_text = (
    "I need to order 10,000 sheets of A4 paper, 5,000 sheets of A3 paper, 100 sheets of 30 lb paper and 500 reams of printer "
    "paper. Also some 24\" monitors, a 3.5 inch drive (15'), and 50% discount. "
    "The supplies must be delivered by April 15, 2025, or 15-08-2023 for our upcoming conference. "
    "Please confirm the order and delivery schedule. (Date of request: 2025-04-04)"
)

count, items = count_order_quantities(order_text)
print("Number of material references:", count)   # -> 3
print("Extracted quantities:", items)            # -> [10000, 5000, 500]


Number of material references: 4
Extracted quantities: [10000, 5000, 100, 500]


In [6]:
len(pd.read_csv("quote_requests_sample.csv"))

20

In [19]:
quote_requests_sample = pd.read_csv("quote_requests_sample.csv")

# Sort by date
quote_requests_sample["request_date"] = pd.to_datetime(
    quote_requests_sample["request_date"], format="%m/%d/%y", errors="coerce"
)
quote_requests_sample = quote_requests_sample.sort_values("request_date")

for idx, row in quote_requests_sample.iterrows():
        request_date:str = row["request_date"].strftime("%Y-%m-%d")  # request_date of type str

        print_rich(f"\n=== Request {idx+1} ===")
        print(f"Context: {row['job']} organizing {row['event']}")
        print(f"Request Date: {request_date}")
        # Process request
        request_with_date = f"{row['request']} (Date of request: {request_date})"
        ############ CONTROL ################
        print_rich(request_with_date, color='bright_green')
        #####################################
        print()
        count, items = count_order_quantities(request_with_date)
        print_rich(f"Number items = {count}, items= {items}", color='bright_magenta')

=== Request 1 ===

Context: office manager organizing ceremony
Request Date: 2025-04-01


I would like to request the following paper supplies for the ceremony: 

- 200 sheets of A4 glossy paper
- 100 sheets of heavy cardstock (white)
- 100 sheets of colored paper (assorted colors)

I need these supplies delivered by April 15, 2025. Thank you. (Date of request: 2025-04-01)

Number items = 3, items= [200, 100, 100]

=== Request 2 ===

...

Number items = 3, items= [2000, 1500, 1000]

=== Request 20 ===

Context: restaurant manager organizing concert
Request Date: 2025-04-17


I would like to place a large order for the following paper supplies for our upcoming concert: 5,000 flyers, 2,000 
posters, and 10,000 tickets. We need these items delivered by May 15, 2025, to ensure adequate time for 
distribution and promotion. Thank you! (Date of request: 2025-04-17)

Number items = 3, items= [5000, 2000, 10000]

In [ ]:
count = 3
error_list =[]
for item in bill_of_materials:
    if item['item'] not in state.available_items:
        error_list.append(item['item'])
check1 = len(error_list) > 0
check2 = len(bill_of_materials) > count

In [24]:
order_text = """[
    {
      "item": "Colored paper",
      "quantity": 500
    },
    {
      "item": "Crepe paper",
      "quantity": 300
    }
]"""


count, items = count_order_quantities(order_text)
print("Number of material references:", count)   # -> 3
print("Extracted quantities:", items)   

Number of material references: 2
Extracted quantities: [500, 300]


In [6]:
def extract_request_date(text: str) -> str:
    """
    Extracts the 'Date of request' from an order string.
    Returns a datetime.date if found, else None.

    Args:
        text: customer order with section 'Date of request: date to extract in the format "%Y-%m-%d"'

    Returns
        extracted date (str) or None
    """
    match = re.search(r'Date of request:\s*([0-9]{4}[-/][0-9]{2}[-/][0-9]{2})', text)
    if match:
        try:
            return match.group(1)
        except ValueError:
            return None
    return None

In [7]:
text = """I would like to place a large order for the following paper supplies for our upcoming concert: 5,000 flyers, 2,000 
posters, and 10,000 tickets. We need these items delivered by May 15, 2025, to ensure adequate time for 
distribution and promotion. Thank you! (Date of request: 2025-04-17)"""

extract_request_date(text)

'2025-04-17'